### Problem Statement

In [3]:
# Total Losses from a Vertical Fixed Roof Tank with Mixutre
# https://www3.epa.gov/ttn/chief/ap42/ch07/final/c07s01.pdf
# Example starting on page 154

In [ ]:
ISSUES:
    - resolve psia vs psig issue with Pint
    - resolve lb-mol issue with Pint (no units for moles?)
    - resolve temperature issue

In [ ]:
NEED TO UPDATE:
    - new references tables from updated new AP42 guidelines

### Imports

In [1]:
import math, pint
import pandas as pd

In [2]:
from pint import UnitRegistry
ureg = UnitRegistry()

# Pint is a Python package to define, operate and manipulate
# physical quantities: the product of a numerical value and 
# a unit of measurement. It allows arithmetic operations 
# between them and conversions from and to different units. 
# It is distributed with a comprehensive list of physical 
# units, prefixes and constants.

# list of units: https://github.com/hgrecco/pint/blob/master/pint/default_en.txt
# to assign unit, variable = magnitude * ureg.unit
# to re-assign units, variable.ito(ureg.unit)
# scale units up or down, i.e., m to km, .to_compact()
# convert to base units, .to_base_units()
# dimensional reduction, .ito_reduced_units()

### Constants

In [4]:
# CONSTANTS

# Ideal Gas Constant, psia*ft^3/lb-mole*°R)
# - psi is only option in pint?
# - need to be able to differentiate between gauage and atmospheric
# psig = psia - 14.7 (units: atmospheres)

# how to address lb-mole units and psia and psig
# R = 10.731 * ureg.psi * ureg.ft**3 / ureg.degR
# R

### User Inputs

In [100]:
# USER INPUTS: Variable - Description (range, units, default)

tank_type = 'vertical'
roof_type = 'cone'

comp1 = ['benzene', 2812/3171]
comp2 = ['toluene', 258/3171]
comp3 = ['cyclohexane', 101/3171]

# Diameter (> 5, ft)
D = 6 * ureg.ft

# Tank Shell Height (5 to 65, ft)
HS = 12 * ureg.ft

# Average Liquid Height
HL = 8 * ureg.ft

# Minimum Liquid Height (ft)
HLN = 11.5 / ureg.ft

# Maximum Liquid Height (ft)
HLX = 4.5 / ureg.ft

# Tank Cone Roof Slope (Cone, ft/ft, 0.0625)
SR = 0.0625 * ureg.ft / ureg.ft

### AP-42 Reference Data

##### Table 7.1-6 PAINT SOLAR ABSORPTANCE

In [116]:
alpha_data = pd.read_csv("ref_docs/ref_csv_files/ap42_ch7_alpha.csv")
alpha_data.head()

,Paint Color,alpha_good,alpha_poor
0,Aluminum Specular,0.39,0.49
1,Aluminum Diffuse,0.60,0.68
2,Aluminum Unpainted,0.10,0.15
3,Beige/Cream,0.35,0.49
4,Black,0.97,0.97


In [130]:
# Tank alpha, dimensionless

# User Inputs
tank_color = 'White'
tank_condition = 'Good'

# Filter Alpha Dataframe for Tank Color
mask_tank_color = (alpha_data['Paint Color'] == 'White')
tank_color_alpha_list = alpha_data.loc[mask_tank_color]


# Select Alpha based on Tank Condition
if tank_condition == 'Good':
    tank_alpha = alpha_data.loc[mask_good]
    tank_alpha = tank_color_alpha_list['alpha_good'].tolist()
    tank_alpha = tank_alpha[0]
    print(tank_alpha)
    
else:
    tank_alpha = alpha_data.loc[mask_good]
    tank_alpha = tank_color_alpha_list['alpha_poor'].tolist()
    tank_alpha = tank_alpha[0]
    print(tank_alpha)

0.17


In [131]:
# Shell Alpha, dimensionless

# User Inputs
shell_color = 'White'
shell_condition = 'Good'

# Filter Alpha Dataframe for Tank Color
mask_shell_color = (alpha_data['Paint Color'] == 'White')
shell_color_alpha_list = alpha_data.loc[mask_shell_color]


# Select Alpha based on Tank Condition
if shell_condition == 'Good':
    shell_alpha = alpha_data.loc[mask_good]
    shell_alpha = shell_color_alpha_list['alpha_good'].tolist()
    shell_alpha = shell_alpha[0]
    print(shell_alpha)
    
else:
    shell_alpha = alpha_data.loc[mask_good]
    shell_alpha = shell_color_alpha_list['alpha_poor'].tolist()
    shell_alpha = shell_alpha[0]
    print(shell_alpha)

0.17


In [132]:
# Note to Table 7.1-6. PAINT SOLAR ABSORPTANCE for alpha equation, pg 102
alpha = (tank_alpha + shell_alpha)/2
alpha

0.17

##### Table 7.1-7 METEOROLOGICAL DATA (TAX, TAN, V, I, PA) FOR SELECTED U.S. LOCATIONS

In [59]:
met_data = pd.read_csv("ref_docs/ref_csv_files/ap42_ch7_met_data.csv")
met_data.head()

,Location,Time_Code,TAN_F,TAX_F,I,v,PA_atm
0,"Amarillo, TX",1,21.7,49.1,960.0,13.6,12.92
1,"Amarillo, TX",2,26.1,53.1,1244.0,13.6,12.92
2,"Amarillo, TX",3,32.0,60.8,1631.0,13.6,12.92
3,"Amarillo, TX",4,42.0,71.0,2019.0,13.6,12.92
4,"Amarillo, TX",5,51.9,79.1,2212.0,13.6,12.92


In [102]:
city = 'Denver'
state = 'CO'
location = city+", "+state
location

'Denver, CO'

In [103]:
# pull in TAX, TAN and I from Table 

mask = (met_data['Location'] == location) & (met_data['Time_Code'] == 'Annual')
location_data = met_data.loc[mask]
location_data

,Location,Time_Code,TAN_F,TAX_F,I,v,PA_atm
220,"Denver, CO",Annual,36.233333,64.258333,1568.583333,8.7,12.122


In [112]:
TAN = location_data['TAN_F'].tolist()
TAN = TAN[0]
# TAN = TAN * ureg.degF
TAX 

64.25833333

In [113]:
TAX = location_data['TAX_F'].tolist()
TAX = TAX[0]
# TAN = TAN * ureg.degF
TAX

64.25833333

In [124]:
# AVERAGE DAILY TOTAL INSOLATION FACTOR, Btu/ft^2*d
I = location_data['I'].tolist()
I = I[0]
I = I * ureg.BTU / ureg.ft**2 / ureg.day
I

1568.583333 <Unit('british_thermal_unit / day / foot ** 2')>

In [111]:
# AVERAGE WIND SPEED, mph
v = location_data['v'].tolist()
v = v[0]
v * ureg.mph

8.7 <Unit('mile_per_hour')>

In [115]:
PA = location_data['PA_atm'].tolist()
PA = PA[0]
PA * ureg.psi

12.122 <Unit('pound_force_per_square_inch')>

### KS, Vented Vapor Space Saturation Factor

### KE, Vapor Space Expansion Factor

### WV, Vapor Density

In [ ]:
# VAPOR DENSITY, 

WV = ((MV * PVA) / (R * TLA))

# MV = 
# PVA = 
# R = 
# TLA = 

### VV, Tank Vapor Space Volume

##### Tank Shell Radius

In [19]:
# TANK SHELL RADIUS, ft
# D = diameter, ft

RS = (1/2) * D
RS

3.0 <Unit('foot')>

##### Tank Roof Height

In [20]:
# Tank Roof Height, ft
# SR = tank cone roof slope, ft/ft
# RS = tank shell radius, ft

HR = SR * RS
HR
# HR = SR * (1/2) * D

0.1875 <Unit('foot')>

##### Roof Outage

In [21]:
# ROOF OUTAGE, ft (Eq. 1-17)
# HR = tank roof height, ft

HRO = (1/3) * HR
HRO.ito_reduced_units()
# HRO = (1/3) * SR * (1/2) * D

##### Vapor Space Outage

In [22]:
# VAPOR SPACE OUTAGE, ft (Eq. 1-16)
# HS = tank shell height, ft
# HL = stock liquid height, ft
# HRO = roof outage, ft

HVO = HS - HL + HRO
HVO
# HVO = HS - HL + (1/3) * SR * (1/2) * D

4.0625 <Unit('foot')>

##### Vapor Space Volume

In [23]:
# TANK VAPOR SPACE VOLUME, ft^3 (Eq. 1-3)
# D = diameter, ft
# HVO = vapor space outage, ft

VV = (((math.pi/4) * D**2) * HVO).round(2)
VV
# VV = (((math.pi/4) * D**2) * (HS - HL + (1/3) * SR * (1/2) * D)).round(2)

114.86 <Unit('foot ** 3')>

### LS, Standing Losses

In [25]:
# STANDING LOSSES (Eq. 1-2)

# LS = 365 * VV * WV * KE * KS

# VV = tank vapor space volume, ft^3
# WV = vapor density, lb/ft3
# KE = vapor space expansion factor, per day
# KS = vented vapor space saturation factor, dimensionless

### VQ, Net Working Loss Throughput

In [ ]:
# VQ = net working loss throughput, ft^3/yr

### KN, Working Loss Turnover Factor

In [ ]:
# KN = working loss turnover factor, dimensionless

### KP, Working Loss Product Factor

In [ ]:
# KP = working loss product factor, dimensionless

### KB, Vent Setting Correction Factor

In [ ]:
# KB = vent setting correction factor, dimensionless

### LW, Working Losses

In [ ]:
# WORKING LOSSES (Eq. 1-35)

LW = VQ * KN * KP * WV * KB

# VQ = net working loss throughput, ft^3/yr
# KN = working loss turnover factor, dimensionless
# KP = working loss product factor, dimensionless
# KB = vent setting correction factor, dimensionless

### LT, Total Losses

In [ ]:
# TOTAL LOSSES (Eq. 1-1)

LT = LS + LW

# LT = total loss, lb/yr
# LS = standing loss, lb/yr
# LW = working loss, lb/yr 

### Output

In [26]:
print ("Tank Shell Radius: ", RS)
print ("RS = (1/2) * D")
print ("RS = ",RS," = (1/2) * ",D)
print (" ")
print ("Tank Roof Height: ", HR)
print ("HR = SR * RS")
print ("HR = ",HR," = ",SR," * ",RS)
print (" ")
print ("Roof Outage: ", HRO)
print ("HRO = (1/3) * HR, (Eq. 1-17)")
print ("HRO = ",HRO," = (1/3) * ",HR)
print (" ")
print ("Vapor Space Outage: ", HVO)
print ("HVO = HS - HL + HRO, (Eq. 1-16)")
print ("HVO = ",HVO," = ",HS," - ",HL," + ",HRO)
print (" ")
print ("Tank Vapor Space Volume: ", VV)
print ("VV = (((math.pi/4) * D**2) * HVO), (Eq. 1-3)")
print ("VV = ",VV," = (((pi/4) * ",D,"**2) * ",HVO)
print (" ")

Tank Shell Radius:  3.0 foot
RS = (1/2) * D
RS =  3.0 foot  = (1/2) *  6 foot
 
Tank Roof Height:  0.1875 foot
HR = SR * RS
HR =  0.1875 foot  =  0.0625 dimensionless  *  3.0 foot
 
Roof Outage:  0.0625 foot
HRO = (1/3) * HR, (Eq. 1-17)
HRO =  0.0625 foot  = (1/3) *  0.1875 foot
 
Vapor Space Outage:  4.0625 foot
HVO = HS - HL + HRO, (Eq. 1-16)
HVO =  4.0625 foot  =  12 foot  -  8 foot  +  0.0625 foot
 
Tank Vapor Space Volume:  114.86 foot ** 3
VV = (((math.pi/4) * D**2) * HVO), (Eq. 1-3)
VV =  114.86 foot ** 3  = (((pi/4) *  6 foot **2) *  4.0625 foot
 
